## Notebook Configuration && Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import itertools
import logging
from tqdm import tqdm

import pandas as pd
from prophet import Prophet

from storesales.constants import (
    SUBMISSIONS_PATH,
    EXTERNAL_TRAIN_PATH,
    EXTERNAL_SAMPLE_SUBMISSION_PATH,
    EXTERNAL_TEST_PATH,
    EXTERNAL_OIL_PATH,
    EXTERNAL_HOLIDAYS_EVENTS_PATH
)

## Load && Prepare Data

In [42]:
original_train_df = pd.read_csv(EXTERNAL_TRAIN_PATH, parse_dates=["date"])
original_train_df.sort_values(by=["date", "store_nbr", "family"], inplace=True)

original_test_df = pd.read_csv(EXTERNAL_TEST_PATH, parse_dates=["date"])

sample_submission_df = pd.read_csv(EXTERNAL_SAMPLE_SUBMISSION_PATH, index_col="id")

oil_df = pd.read_csv(EXTERNAL_OIL_PATH, parse_dates=["date"])
oil_df.set_index("date", inplace=True)
oil_df = oil_df.asfreq("D")
oil_df["dcoilwtico"] = oil_df["dcoilwtico"].ffill()
oil_df = oil_df.dropna()

holidays_df = pd.read_csv(EXTERNAL_HOLIDAYS_EVENTS_PATH, parse_dates=["date"])

In [44]:
train_period = original_train_df.index.unique()

## Facebook Prophet

## Store nbr '1' & Family 'GROCERY I'

In [59]:
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
import optuna
from prophet.diagnostics import cross_validation, performance_metrics

In [60]:
def rmsle(y_true, y_pred):
    return np.sqrt(np.mean(np.square(np.log1p(y_true) - np.log1p(y_pred))))

In [108]:
train_df = original_train_df[["date", "sales", "store_nbr", "family"]].copy()
train_df.rename(columns={"date": "ds", "sales": "y"}, inplace=True)

In [109]:
families = original_train_df["family"].unique()
stores = original_train_df["store_nbr"].unique()

In [110]:
train_df = train_df.merge(oil_df, left_on="ds", right_index=True, how="left")

In [119]:
train_df.head()

,ds,y,store_nbr,family,dcoilwtico
1782,2013-01-02,2.000,1,AUTOMOTIVE,93.14
2778,2013-01-02,0.000,37,CELEBRATION,93.14
2777,2013-01-02,554.995,37,BREAD/BAKERY,93.14
2776,2013-01-02,0.000,37,BOOKS,93.14
2775,2013-01-02,1119.000,37,BEVERAGES,93.14


In [111]:
train_df.dropna(inplace=True)

In [112]:
train_df.sort_values(by="ds", inplace=True)

In [113]:
holidays_df = holidays_df[holidays_df["transferred"] == False]

In [114]:
prophet_holidays_df = holidays_df[["date", "description"]].rename(columns={"date": "ds", "description": "holiday"})

In [146]:
class DailyMeanModel:
    def __init__(self, window: int):
        self.window = window
        self.prediction = None
        
    def fit(self, train: pd.DataFrame) -> None:
        self.prediction = train["y"].tail(self.window).mean()
        
    def predict(self, future: pd.DataFrame) -> pd.DataFrame:
        if self.prediction is None:
            raise ValueError("Model not fitted")
        future["yhat"] = [self.prediction] * len(future)
        return future


class DailyMeanModelWrapper:
    def __init__(self): ...
    
    @staticmethod
    def get_model(**kwargs) -> DailyMeanModel:
        return DailyMeanModel(**kwargs)
    
    def objective(self, trial: optuna.Trial, df: pd.DataFrame) -> float:
        model = self.get_model(window=trial.suggest_int("window", 3, 100))
    
        cutoffs = df["ds"].iloc[[-180, -49, -33, -17]].reset_index(drop=True)
        losses = [] 
        for cutoff in cutoffs:
            train_condition = (df["ds"] < cutoff)
            train = df[train_condition]
            future = df[~train_condition][:16]
                        
            model.fit(train)
            forecast = model.predict(future)
            
            y_pred = forecast["yhat"].values
            y_true = forecast["y"].values
            
            losses.append(rmsle(y_true, y_pred))
        
        return np.mean(losses)
    
    
class ProphetWrapper:
    def __init__(self, extra_regressors: list[str] = None, holidays=None):
        self.extra_regressors = extra_regressors
        
        self.holidays = holidays
        self.daily_seasonality = False
        
    def get_model(self, **kwargs) -> Prophet:
        prophet = Prophet(
            daily_seasonality=self.daily_seasonality,
            holidays=self.holidays,
            **kwargs
        )
        
        if self.extra_regressors is not None:
            for regressor in self.extra_regressors:
                prophet.add_regressor(regressor)
        
        return prophet
    
    def objective(self, trial: optuna.Trial, train: pd.DataFrame) -> float:    
        model = self.get_model(            
                yearly_seasonality=trial.suggest_categorical("yearly_seasonality", [True, False]),
                weekly_seasonality=trial.suggest_categorical("weekly_seasonality", [True, False]),
                changepoint_prior_scale=trial.suggest_float("changepoint_prior_scale", 0.001, 0.5),
                seasonality_prior_scale=trial.suggest_int("seasonality_prior_scale", 1, 20),
                seasonality_mode=trial.suggest_categorical("seasonality_mode", ["additive", "multiplicative"])
        ).fit(train)
        
        cutoffs = train["ds"].iloc[[-180, -49, -33, -17]].reset_index(drop=True)
        df_cv = cross_validation(
            model, 
            horizon="16 days", 
            initial="1095 days", 
            cutoffs=cutoffs, 
            disable_tqdm=True, 
            parallel="processes"
        )
        loss = df_cv.groupby("cutoff")[["y", "yhat"]].apply(lambda group: rmsle(group["y"], group["yhat"])).mean()
        return loss
    

class SalesPredictor:
    def __init__(self, prophet_wrapper: ProphetWrapper, daily_wrapper: DailyMeanModel):
        self.best_models = {}
        self.models = {
            DailyMeanModel.__name__: daily_wrapper,
            ProphetWrapper.__name__: prophet_wrapper
        }
    
    def fit(self, train: pd.DataFrame) -> None:
        for model in self.models:
            model.fit(train)
            
    def objective(self, trial: optuna.Trial, train: pd.DataFrame) -> float:
        model_name = trial.suggest_categorical("model", self.models.keys())
        return self.models[model_name].objective(trial, train)
            
    def get_best_model(self, best_params) -> DailyMeanModel | Prophet:
        model_name = best_params.pop("model")
        return self.models[model_name].get_model(**best_params)
    
    def save_best(self, best_params, loss) -> None:
        model_name = best_params.pop("model")
        self.best_models[model_name] = {"params": best_params, "loss": loss}
        

In [151]:
optuna.logging.set_verbosity(optuna.logging.WARN)
logging.getLogger("cmdstanpy").setLevel(logging.WARNING)

def run_study(df: pd.DataFrame):
    daily_wrapper = DailyMeanModelWrapper()
    prophet_wrapper = ProphetWrapper(extra_regressors=["dcoilwtico"], holidays=prophet_holidays_df)
    
    predictor = SalesPredictor(daily_wrapper=daily_wrapper, prophet_wrapper=prophet_wrapper)
    
    for family in tqdm(families[:3]):
        for store in np.random.choice(stores, size=3, replace=False):
            train = df[(df["store_nbr"] == store) & (df["family"] == family)][["ds", "y", "dcoilwtico"]].copy()
            
            outer_results = []
            outer_tscv = TimeSeriesSplit(n_splits=4, test_size=16)
            for outer_train_index, outer_test_index in outer_tscv.split(train):
                outer_train, outer_test = train.iloc[outer_train_index], train.iloc[outer_test_index]
            
                study = optuna.create_study(direction="minimize")
                study.optimize(lambda trial: predictor.objective(trial, outer_train), n_trials=5, show_progress_bar=True) 
                
                best_model = predictor.get_best_model(study.best_params)
                best_model.fit(outer_train)
                            
                future = outer_test[["ds", "dcoilwtico"]]
                
                forecast = best_model.predict(future)
                
                y_pred = forecast["yhat"].values
                y_true = outer_test["y"].values
                
                outer_rmsle = rmsle(y_true, y_pred)
                outer_results.append(outer_rmsle)
            
                # print(f"Outer RMSLE: {outer_rmsle}")
                # print(f"best params: {study.best_params}")

            final_outer_rmsle = np.mean(outer_results)
            print(f"\n\nFinal Outer RMSLE: {final_outer_rmsle}")

In [152]:
run_study(train_df)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

...

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 3/3 [05:37<00:00, 112.50s/it]



Final Outer RMSLE: 0.43098465511806316


In [ ]:
0.126081053304793